In [1]:
import torch
from torch_geometric.data import Data
# from torch_geometric.transforms import KNNGraph
from torch_geometric.nn import knn_graph
from torch_geometric.nn import knn
import pandas as pd

In [ ]:
df_startup = pd.read_csv('startup_features.csv')
df_org_des = pd.read_csv('organization_descriptions.csv')
df_org = pd.read_csv('organizations.csv')
df = pd.read_csv('investor_startup_rel.csv')
df_startup.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
df_org_des  = pd.merge(df[['org_uuid','org_name']],df_org_des[['uuid','description']],how='left', left_on='org_uuid',right_on='uuid')
df_org_des  = pd.merge(df_org_des[['org_uuid','org_name','description']],df_org[['uuid','short_description']],how='left', left_on='org_uuid',right_on='uuid')
df_org_des

,org_uuid,org_name,description,uuid,short_description
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH,Formel D GmbH is a automotive manufacturer and...,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D is the global service provider of the...
1,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,"Resilio, a digital well-being company, is comb...",000014da-0c46-b9cb-0941-3a93c027b119,App-based resilience training designed for work
2,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,"Resilio, a digital well-being company, is comb...",000014da-0c46-b9cb-0941-3a93c027b119,App-based resilience training designed for work
3,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,"Resilio, a digital well-being company, is comb...",000014da-0c46-b9cb-0941-3a93c027b119,App-based resilience training designed for work
4,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,"Resilio, a digital well-being company, is comb...",000014da-0c46-b9cb-0941-3a93c027b119,App-based resilience training designed for work
...,...,...,...,...,...
354643,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,ERNIT is a mobile application platform that pr...,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...
354644,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,ERNIT is a mobile application platform that pr...,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...
354645,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,ERNIT is a mobile application platform that pr...,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...
354646,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,ERNIT is a mobile application platform that pr...,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...


In [ ]:
df_org_des.isna().sum()

org_uuid                 0
org_name                 0
description          19579
uuid                     0
short_description        9
dtype: int64

In [ ]:
df_org_des['description'] = df_org_des['description'].fillna(df_org_des['short_description'])
df_org_des['description'] = df_org_des['description'].fillna(df_org_des['org_name'])

In [ ]:
df_org_des = df_org_des.drop(columns=['short_description','org_name','uuid'], axis=1)

df_org_des.isna().sum()

org_uuid       0
description    0
uuid           0
dtype: int64

In [ ]:
df_org_des

,org_uuid,description
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...
1,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb..."
2,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb..."
3,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb..."
4,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb..."
...,...,...
354643,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...
354644,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...
354645,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...
354646,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Generate embeddings for each description in batches
embeddings = model.encode(df_org_des['description'].tolist(), convert_to_tensor=True)

# Add embeddings to DataFrame
df_org_des['embeddings'] = embeddings.tolist()


In [ ]:
df_org_des

,org_uuid,description,embeddings
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,"[-0.1373351514339447, 0.015541150234639645, 0...."
1,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.02787846513092518, -0.0029251486994326115,..."
2,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.027878431603312492, -0.0029250988736748695..."
3,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.027878431603312492, -0.0029250988736748695..."
4,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.02787846513092518, -0.0029251486994326115,..."
...,...,...,...
354643,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787035942077637, 0.054107166826725006, ..."
354644,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787035942077637, 0.054107166826725006, ..."
354645,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787035942077637, 0.054107166826725006, ..."
354646,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787045255303383, 0.05410715192556381, -..."


In [ ]:
embeddings_df = pd.DataFrame(embeddings.cpu())

# Concatenate the original DataFrame with the embeddings DataFrame
df_org_des = pd.concat([df_org_des.reset_index(drop=True), embeddings_df.reset_index(drop=True)], axis=1)

In [ ]:
df_org_des

,org_uuid,description,embeddings,0,1,2,3,4,5,6,...,374,375,376,377,378,379,380,381,382,383
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,"[-0.1373351514339447, 0.015541150234639645, 0....",-0.137335,0.015541,0.045160,-0.052872,-0.090663,0.000803,0.019339,...,-0.060487,0.020525,-0.028828,-0.012750,-0.016189,-0.031651,-0.005810,0.006511,0.130285,0.056890
1,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.02787846513092518, -0.0029251486994326115,...",-0.027878,-0.002925,-0.024699,0.042006,0.010432,-0.010297,0.038107,...,0.067666,-0.011182,0.036096,0.069602,-0.054181,0.037838,0.060975,0.051151,0.015449,0.014736
2,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.027878431603312492, -0.0029250988736748695...",-0.027878,-0.002925,-0.024699,0.042006,0.010432,-0.010297,0.038107,...,0.067666,-0.011182,0.036096,0.069602,-0.054181,0.037838,0.060975,0.051151,0.015449,0.014736
3,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.027878431603312492, -0.0029250988736748695...",-0.027878,-0.002925,-0.024699,0.042006,0.010432,-0.010297,0.038107,...,0.067666,-0.011182,0.036096,0.069602,-0.054181,0.037838,0.060975,0.051151,0.015449,0.014736
4,000014da-0c46-b9cb-0941-3a93c027b119,"Resilio, a digital well-being company, is comb...","[-0.02787846513092518, -0.0029251486994326115,...",-0.027878,-0.002925,-0.024699,0.042006,0.010432,-0.010297,0.038107,...,0.067666,-0.011182,0.036096,0.069602,-0.054181,0.037838,0.060975,0.051151,0.015449,0.014736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354643,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787035942077637, 0.054107166826725006, ...",-0.008787,0.054107,-0.047730,-0.012193,0.036330,-0.080591,0.078695,...,0.019096,0.017681,0.032599,-0.030828,0.025052,0.104787,0.047748,-0.029783,0.064655,0.066431
354644,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787035942077637, 0.054107166826725006, ...",-0.008787,0.054107,-0.047730,-0.012193,0.036330,-0.080591,0.078695,...,0.019096,0.017681,0.032599,-0.030828,0.025052,0.104787,0.047748,-0.029783,0.064655,0.066431
354645,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787035942077637, 0.054107166826725006, ...",-0.008787,0.054107,-0.047730,-0.012193,0.036330,-0.080591,0.078695,...,0.019096,0.017681,0.032599,-0.030828,0.025052,0.104787,0.047748,-0.029783,0.064655,0.066431
354646,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT is a mobile application platform that pr...,"[-0.008787045255303383, 0.05410715192556381, -...",-0.008787,0.054107,-0.047730,-0.012193,0.036330,-0.080591,0.078695,...,0.019096,0.017681,0.032599,-0.030828,0.025052,0.104787,0.047748,-0.029783,0.064655,0.066431


In [ ]:
df_org_des.drop_duplicates(inplace = True)

In [ ]:
df_org_des.to_csv('startup_text_embedding.csv')

In [ ]:
df_org_des = pd.read_csv('startup_text_embedding.csv')
df_startup = pd.read_csv('startup_features.csv')
df = pd.read_csv('investor_startup_rel_freq.csv')
df_startup.drop('Unnamed: 0',inplace=True,axis=1)
df_org_des.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
df_org_des = pd.merge(df['org_uuid'],df_org_des,how='left', left_on='org_uuid',right_on='org_uuid')

In [ ]:
node_features = df_org_des.loc[df_org_des['org_uuid'].drop_duplicates().index].drop(columns=['org_uuid','description','embeddings'])

In [ ]:
df_startup = pd.merge(df_org_des,df_startup,how='left', left_on='org_uuid',right_on='org_uuid')

In [ ]:
org_features_df = df_startup.loc[df_startup['org_uuid'].drop_duplicates().index].drop(columns=['org_uuid','description','embeddings'])

In [ ]:
df_startup.drop_duplicates(inplace = True)

In [ ]:
df_startup.to_csv('startup_features_n_text_embedding.csv')